In [28]:
import os

# 1. FORCE the CUDA path
os.environ["CUDA_PATH"] = "/usr/local/cuda"

# 2. Now import flamegpu
import pyflamegpu

In [29]:
%%writefile repulsion.cu
#include "flamegpu/flamegpu.h"

// Function 1: Output Message
// Input: None, Output: MessageSpatial2D
FLAMEGPU_AGENT_FUNCTION(output_message, flamegpu::MessageNone, flamegpu::MessageSpatial2D) {
    FLAMEGPU->message_out.setVariable<flamegpu::id_t>("id", FLAMEGPU->getID());
    FLAMEGPU->message_out.setLocation(
        FLAMEGPU->getVariable<float>("x"),
        FLAMEGPU->getVariable<float>("y")
    );
    return flamegpu::ALIVE;
}

// Function 2: Move Response
// Input: MessageSpatial2D, Output: None
FLAMEGPU_AGENT_FUNCTION(move_response, flamegpu::MessageSpatial2D, flamegpu::MessageNone) {
    float fx = 0.0;
    float fy = 0.0;
    const float my_x = FLAMEGPU->getVariable<float>("x");
    const float my_y = FLAMEGPU->getVariable<float>("y");

    // Loop through neighbors
    for (const auto &message : FLAMEGPU->message_in(my_x, my_y)) {
        if (message.getVariable<flamegpu::id_t>("id") != FLAMEGPU->getID()) {
            float dx = my_x - message.getVariable<float>("x");
            float dy = my_y - message.getVariable<float>("y");
            float dist = sqrt(dx*dx + dy*dy);

            if (dist > 0.0f) {
                float force = 0.01f / (dist * dist);
                if (force > 0.5f) force = 0.5f;

                fx += (dx / dist) * force;
                fy += (dy / dist) * force;
            }
        }
    }

    FLAMEGPU->setVariable<float>("x", my_x + fx);
    FLAMEGPU->setVariable<float>("y", my_y + fy);

    return flamegpu::ALIVE;
}

Overwriting repulsion.cu


In [30]:
%%writefile sim.py
import pyflamegpu
import random
import matplotlib.pyplot as plt

# --- 1. DEFINE MODEL ---
model = pyflamegpu.ModelDescription("SocialDistancing")

# Define Message (Spatial 2D = Efficient Neighbor Search)
msg = model.newMessageSpatial2D("location_msg")
msg.newVariableID("id")
msg.setRadius(2.0)     # Agents only see neighbors within 2m
msg.setMin(0, 0)       # Bounds of the world (0,0) to (10,10)
msg.setMax(10, 10)

# Define Agent
agent = model.newAgent("Person")
agent.newVariableFloat("x")
agent.newVariableFloat("y")

# Load C++ Logic
with open("repulsion.cu") as f:
    cpp_code = f.read()

# Attach Functions to Agent
fn_out = agent.newRTCFunction("output_message", cpp_code)
fn_out.setMessageOutput("location_msg")

fn_move = agent.newRTCFunction("move_response", cpp_code)
fn_move.setMessageInput("location_msg")

# Execution Order (Layer 1: Send Msg, Layer 2: Read & Move)
model.newLayer().addAgentFunction(fn_out)
model.newLayer().addAgentFunction(fn_move)

# --- 2. INITIALIZE POPULATION ---
simulation = pyflamegpu.CUDASimulation(model)
simulation.SimulationConfig().steps = 50  # Run for 50 frames

# Create 100 agents clumping in the center (5,5)
population = pyflamegpu.AgentVector(agent, 100)
for i in range(100):
    instance = population[i]
    instance.setVariableFloat("x", 5.0 + random.uniform(-0.5, 0.5))
    instance.setVariableFloat("y", 5.0 + random.uniform(-0.5, 0.5))

simulation.setPopulationData(population)

# --- 3. RUN & PLOT ---
print("Running simulation...")
simulation.simulate()

# Fetch data back to CPU
simulation.getPopulationData(population)

# Plotting with Matplotlib (Since we are headless)
x_vals = [population[i].getVariableFloat("x") for i in range(100)]
y_vals = [population[i].getVariableFloat("y") for i in range(100)]

plt.figure(figsize=(6,6))
plt.scatter(x_vals, y_vals, c='blue', alpha=0.6)
plt.xlim(0, 10); plt.ylim(0, 10)
plt.title("After 50 Steps: Social Distancing!")
plt.grid(True)
plt.show()

Overwriting sim.py


In [31]:
# 1. Install the package
# Tell pip to look at the FlameGPU server for the files
%pip install pyflamegpu --extra-index-url https://whl.flamegpu.com/whl/cuda120/

# 2. (Crucial) Check that Colab sees your GPU
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://whl.flamegpu.com/whl/cuda120/
Sat Dec 20 15:49:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             27W /   70W |     104MiB /  15360MiB |      0%      Default |
|                                         |              

In [32]:
import pyflamegpu

# 1. Define the model
model = pyflamegpu.ModelDescription("SmokeTest")
agent = model.newAgent("Agent")
agent.newVariableInt("x") # Just giving the agent some memory

# 2. Build the simulation (Using the NEW class name)
try:
    # ❌ OLD WAY: simulation = pyflamegpu.CUDAAgentModel(model)
    # ✅ NEW WAY:
    simulation = pyflamegpu.CUDASimulation(model)
    
    print("✅ SUCCESS: FlameGPU is installed and talking to the GPU!")
    
except AttributeError:
    print("❌ ERROR: Class name mismatch. Try 'CUDASimulation'.")
except Exception as e:
    print(f"❌ ERROR: Something else broke.\n{e}")


✅ SUCCESS: FlameGPU is installed and talking to the GPU!


In [33]:
!python sim.py

Traceback (most recent call last):
  File "/content/sim.py", line 29, in <module>
    fn_move.setMessageInput("location_msg")
  File "/usr/local/lib/python3.12/dist-packages/pyflamegpu/pyflamegpu.py", line 8101, in setMessageInput
    return _pyflamegpu.AgentFunctionDescription_setMessageInput(self, *args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyflamegpu.pyflamegpu.FLAMEGPURuntimeException: (InvalidMessageType) /__w/FLAMEGPU2/FLAMEGPU2/src/flamegpu/model/AgentFunctionDescription.cpp(219): Message ('location_msg') type 'MessageSpatial2D' does not match type 'MessageNone' applied to FLAMEGPU_AGENT_FUNCTION ('move_response'), in AgentFunctionDescription::setMessageInput().


In [34]:
%%writefile all_in_one.py

import pyflamegpu
import random
import math

import pyflamegpu
import random
import math
import matplotlib.pyplot as plt

# --- 1. THE C++ CODE ---
# FIX: We use fully qualified names (flamegpu::...) so the parser finds them.
# FIX: Renamed functions to _v3 to ensure a fresh compile.
cpp_code = """
#include "flamegpu/flamegpu.h"

// Output Function
FLAMEGPU_AGENT_FUNCTION(output_message_v3, flamegpu::MessageNone, flamegpu::MessageSpatial2D) {
    FLAMEGPU->message_out.setVariable<flamegpu::id_t>("id", FLAMEGPU->getID());
    FLAMEGPU->message_out.setLocation(
        FLAMEGPU->getVariable<float>("x"),
        FLAMEGPU->getVariable<float>("y")
    );
    return flamegpu::ALIVE;
}

// Move Function
// FIX: Explicitly use flamegpu::MessageSpatial2D here so the RTC parser sees it
FLAMEGPU_AGENT_FUNCTION(move_response_v3, flamegpu::MessageSpatial2D, flamegpu::MessageNone) {
    float fx = 0.0;
    float fy = 0.0;
    const float my_x = FLAMEGPU->getVariable<float>("x");
    const float my_y = FLAMEGPU->getVariable<float>("y");

    // Loop through neighbors
    for (const auto &message : FLAMEGPU->message_in(my_x, my_y)) {
        if (message.getVariable<flamegpu::id_t>("id") != FLAMEGPU->getID()) {
            float dx = my_x - message.getVariable<float>("x");
            float dy = my_y - message.getVariable<float>("y");
            float dist = sqrt(dx*dx + dy*dy);

            if (dist > 0.0f) {
                float force = 0.01f / (dist * dist);
                if (force > 0.5f) force = 0.5f;

                fx += (dx / dist) * force;
                fy += (dy / dist) * force;
            }
        }
    }

    FLAMEGPU->setVariable<float>("x", my_x + fx);
    FLAMEGPU->setVariable<float>("y", my_y + fy);

    return flamegpu::ALIVE;
}
"""

# --- 2. PYTHON SETUP ---
model = pyflamegpu.ModelDescription("SocialDistancing_v3")

# Define Message
msg = model.newMessageSpatial2D("location_msg")
msg.newVariableID("id")
msg.setRadius(2.0)
msg.setMin(0, 0)
msg.setMax(10, 10)

# Define Agent
agent = model.newAgent("Person")
agent.newVariableFloat("x")
agent.newVariableFloat("y")

# Attach C++ Functions
# IMPORTANT: Use the new _v3 names
fn_out = agent.newRTCFunction("output_message_v3", cpp_code)
fn_out.setMessageOutput("location_msg")

fn_move = agent.newRTCFunction("move_response_v3", cpp_code)
fn_move.setMessageInput("location_msg")

# Layering
model.newLayer().addAgentFunction(fn_out)
model.newLayer().addAgentFunction(fn_move)

# --- 3. RUN ---
simulation = pyflamegpu.CUDASimulation(model)
simulation.SimulationConfig().steps = 50

# Create 100 agents in a clump
population = pyflamegpu.AgentVector(agent, 100)
for i in range(100):
    instance = population[i]
    instance.setVariableFloat("x", 5.0 + random.uniform(-0.5, 0.5))
    instance.setVariableFloat("y", 5.0 + random.uniform(-0.5, 0.5))

simulation.setPopulationData(population)

print("🚀 Attempting to run simulation v3...")
simulation.simulate()
print("✅ Simulation Finished!")

# --- 4. PLOT ---
simulation.getPopulationData(population)
x_vals = [population[i].getVariableFloat("x") for i in range(100)]
y_vals = [population[i].getVariableFloat("y") for i in range(100)]

plt.figure(figsize=(6,6))
plt.scatter(x_vals, y_vals, c='blue', alpha=0.6)
plt.xlim(0, 10); plt.ylim(0, 10)
plt.title("After 50 Steps: Agents Spread Out")
plt.show()

Overwriting all_in_one.py


In [35]:
!python all_in_one.py

Traceback (most recent call last):
  File "/content/all_in_one.py", line 80, in <module>
    fn_move.setMessageInput("location_msg")
  File "/usr/local/lib/python3.12/dist-packages/pyflamegpu/pyflamegpu.py", line 8101, in setMessageInput
    return _pyflamegpu.AgentFunctionDescription_setMessageInput(self, *args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyflamegpu.pyflamegpu.FLAMEGPURuntimeException: (InvalidMessageType) /__w/FLAMEGPU2/FLAMEGPU2/src/flamegpu/model/AgentFunctionDescription.cpp(219): Message ('location_msg') type 'MessageSpatial2D' does not match type 'MessageNone' applied to FLAMEGPU_AGENT_FUNCTION ('move_response_v3'), in AgentFunctionDescription::setMessageInput().


In [36]:
%%writefile simple_test.py 

import os
import sys
import glob

# --- 🛠️ FIX: AUTO-DETECT C++ INCLUDE PATHS ---
# We hunt for the C++ standard library folders on Colab
include_paths = []

# 1. Find the main C++ include directory (e.g., /usr/include/c++/11)
cpp_roots = glob.glob("/usr/include/c++/*")
if cpp_roots:
    # Sort to get the latest version
    latest_cpp = sorted(cpp_roots)[-1]
    include_paths.append(latest_cpp)
    # Also add the architecture-specific folder (x86_64-linux-gnu)
    # This is crucial for bits/c++config.h
    include_paths.append(f"/usr/include/x86_64-linux-gnu/c++/{os.path.basename(latest_cpp)}")

# 2. Add CUDA includes just in case
include_paths.append("/usr/local/cuda/include")

# 3. Format them as a colon-separated string
# We set BOTH standard variables to cover all bases
full_path_str = ":".join(include_paths)
os.environ["CPLUS_INCLUDE_PATH"] = full_path_str
os.environ["C_INCLUDE_PATH"] = full_path_str
os.environ["CUDA_PATH"] = "/usr/local/cuda"

print(f"🔧 Injected Include Paths: {full_path_str}")

# --- NOW we import flamegpu ---
import pyflamegpu
import math
import matplotlib.pyplot as plt

# --- THE SIMULATION CODE ---
cpp_code = """
#include "flamegpu/flamegpu.h"

FLAMEGPU_AGENT_FUNCTION(move_circle, flamegpu::MessageNone, flamegpu::MessageNone) {
    float x = FLAMEGPU->getVariable<float>("x");
    float y = FLAMEGPU->getVariable<float>("y");
    
    // Rotate
    float angle = 0.1f;
    float new_x = x * cos(angle) - y * sin(angle);
    float new_y = x * sin(angle) + y * cos(angle);
    
    FLAMEGPU->setVariable<float>("x", new_x);
    FLAMEGPU->setVariable<float>("y", new_y);
    
    return flamegpu::ALIVE;
}
"""

model = pyflamegpu.ModelDescription("SimpleTest_Fixed")
agent = model.newAgent("Agent")
agent.newVariableFloat("x")
agent.newVariableFloat("y")

func = agent.newRTCFunction("move_circle", cpp_code)
model.newLayer().addAgentFunction(func)

try:
    simulation = pyflamegpu.CUDASimulation(model)
    simulation.SimulationConfig().steps = 20

    population = pyflamegpu.AgentVector(agent, 50)
    for i in range(50):
        p = population[i]
        p.setVariableFloat("x", float(i) * 0.2)
        p.setVariableFloat("y", 0.0)

    simulation.setPopulationData(population)

    print("🚀 Running Simple Test...")
    simulation.simulate()
    print("✅ SUCCESS: The kernel executed!")
    
    # Plot
    simulation.getPopulationData(population)
    x = [population[i].getVariableFloat("x") for i in range(50)]
    y = [population[i].getVariableFloat("y") for i in range(50)]
    plt.scatter(x, y); plt.title("It works!"); plt.show()

except Exception as e:
    print(f"❌ FAIL: {e}")

Overwriting simple_test.py


In [37]:
!python3 simple_test.py

🔧 Injected Include Paths: /usr/include/c++/11:/usr/include/x86_64-linux-gnu/c++/11:/usr/local/cuda/include
Failed to load program for agent function (condition) 'move_circle', log:
Compilation failed: NVRTC_ERROR_COMPILATION
__jitify_rel_inc@__jitify_I0@/flamegpu@__jitify_name@flamegpu/runtime/HostAPI.h(8): catastrophic error: could not open source file "unordered_map" (no directories in search list)
  #include <unordered_map>
                          ^

1 catastrophic error detected in the compilation of "move_circle_program".
Compilation terminated.
❌ FAIL: (InvalidAgentFunc) /__w/FLAMEGPU2/FLAMEGPU2/src/flamegpu/detail/JitifyCache.cu(461): Error loading agent function (or function condition) ('move_circle'): function had compilation errors:
Compilation failed: NVRTC_ERROR_COMPILATION
__jitify_rel_inc@__jitify_I0@/flamegpu@__jitify_name@flamegpu/runtime/HostAPI.h(8): catastrophic error: could not open source file "unordered_map" (no directories in search list)
  #include <unordered_

In [38]:
import os
import sys

# --- FIX: TELL FLAMEGPU WHERE CUDA IS ---
# This is the magic key that was missing
os.environ["CUDA_PATH"] = "/usr/local/cuda"

import pyflamegpu
import math
import matplotlib.pyplot as plt

# --- 1. THE C++ KERNEL ---
cpp_code = """
#include "flamegpu/flamegpu.h"

FLAMEGPU_AGENT_FUNCTION(move_circle, flamegpu::MessageNone, flamegpu::MessageNone) {
    float x = FLAMEGPU->getVariable<float>("x");
    float y = FLAMEGPU->getVariable<float>("y");
    
    // Rotate position
    float angle = 0.1f;
    float new_x = x * cos(angle) - y * sin(angle);
    float new_y = x * sin(angle) + y * cos(angle);
    
    FLAMEGPU->setVariable<float>("x", new_x);
    FLAMEGPU->setVariable<float>("y", new_y);
    
    return flamegpu::ALIVE;
}
"""

# --- 2. PYTHON SETUP ---
model = pyflamegpu.ModelDescription("SimpleTest_Fixed")
agent = model.newAgent("Agent")
agent.newVariableFloat("x")
agent.newVariableFloat("y")

func = agent.newRTCFunction("move_circle", cpp_code)
model.newLayer().addAgentFunction(func)

# --- 3. RUN ---
try:
    simulation = pyflamegpu.CUDASimulation(model)
    simulation.SimulationConfig().steps = 20

    population = pyflamegpu.AgentVector(agent, 50)
    for i in range(50):
        p = population[i]
        p.setVariableFloat("x", float(i) * 0.2)
        p.setVariableFloat("y", 0.0)

    simulation.setPopulationData(population)

    print("🚀 Running Simple Test (With CUDA_PATH fixed)...")
    simulation.simulate()
    print("✅ SUCCESS: It worked!")
    
    # Quick plot to prove it
    simulation.getPopulationData(population)
    x = [population[i].getVariableFloat("x") for i in range(50)]
    y = [population[i].getVariableFloat("y") for i in range(50)]
    plt.scatter(x, y); plt.title("Success!"); plt.show()

except Exception as e:
    print(f"❌ FAIL: {e}")

❌ FAIL: (InvalidAgentFunc) /__w/FLAMEGPU2/FLAMEGPU2/src/flamegpu/detail/JitifyCache.cu(461): Error loading agent function (or function condition) ('move_circle'): function had compilation errors:
Compilation failed: NVRTC_ERROR_COMPILATION
__jitify_rel_inc@__jitify_I0@/flamegpu@__jitify_name@flamegpu/runtime/HostAPI.h(8): catastrophic error: could not open source file "unordered_map" (no directories in search list)
  #include <unordered_map>
                          ^

1 catastrophic error detected in the compilation of "move_circle_program".
Compilation terminated.

